In [1]:
import torch
from map_action_tranform import get_transform
from map_action_data_loader import map_action_test_data_loader, map_action_test_dataset 
from m_a_detection_model import map_action_instance_segmentation_model
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, accuracy_score, average_precision_score
from sklearn.metrics import precision_recall_curve
import numpy as np
import matplotlib.pyplot as plt


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [2]:
# Function to calculate IoU
def calculate_iou(pred_boxes, true_boxes):
    x1 = torch.max(pred_boxes[:, 0], true_boxes[:, 0])
    y1 = torch.max(pred_boxes[:, 1], true_boxes[:, 1])
    x2 = torch.min(pred_boxes[:, 2], true_boxes[:, 2])
    y2 = torch.min(pred_boxes[:, 3], true_boxes[:, 3])

    intersection = torch.clamp(x2 - x1, min=0) * torch.clamp(y2 - y1, min=0)
    area_pred = (pred_boxes[:, 2] - pred_boxes[:, 0]) * (pred_boxes[:, 3] - pred_boxes[:, 1])
    area_true = (true_boxes[:, 2] - true_boxes[:, 0]) * (true_boxes[:, 3] - true_boxes[:, 1])
    union = area_pred + area_true - intersection

    iou = intersection / union
    return iou

In [7]:
# Testing pipeline


def test_model(model, test_loader, device):
    model.eval()

    all_pred_boxes = []
    all_true_boxes = []

    for images, targets in test_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        with torch.no_grad():
            outputs = model(images)

        for output, target in zip(outputs, targets):
            pred_boxes = output['boxes'].cpu()
            true_boxes = target['boxes'].cpu()
            all_pred_boxes.append(pred_boxes)
            all_true_boxes.append(true_boxes)

    all_pred_boxes = torch.cat(all_pred_boxes, dim=0)
    all_true_boxes = torch.cat(all_true_boxes, dim=0)

    ious = calculate_iou(all_pred_boxes, all_true_boxes)

    # Threshold for considering a detection correct
    detection_threshold = 0.5
    pred_labels = (ious > detection_threshold).float()

    # Convert ious to binary predictions
    binary_ious = (ious > detection_threshold).float()

    # Calculate average precision using binary predictions
    
    average_precision = average_precision_score(np.ones(len(binary_ious)), binary_ious) 

    true_labels = torch.ones(len(all_true_boxes))
    
    # ...
    true_labels = torch.ones(len(all_true_boxes))
    print(f'Length of true_labels: {len(true_labels)}')
    print(f'Length of pred_labels: {len(pred_labels)}')
    accuracy = accuracy_score(true_labels, pred_labels)
    # ...

    accuracy = accuracy_score(true_labels, pred_labels)
    precision = precision_score(true_labels, pred_labels)
    
    print(f'Mean Average Precision (mAP): {average_precision:.4f}')
    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')

    # Plotting precision-recall curve
    precision, recall, _ = precision_recall_curve(true_labels, ious)
    plt.plot(recall, precision, marker='.')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.show()

In [8]:
root_dir_test = '/home/mapaction/Documents/Exp-data/Coco/images/'
annotation_file_test = '/home/mapaction/Documents/Exp-data/Coco/result.json'

# Load the pre-trained model
model_path = '/home/mapaction/mapaction_env/Map-Action-Model/model/MAISM1.pth'
model = map_action_instance_segmentation_model(2)  # Initialize your model architecture
model.load_state_dict(torch.load(model_path))
model.eval()

# Create the testing dataset
test_transform = get_transform
test_dataset = map_action_test_dataset
  # Replace with your class names

# Create a data loader for testing
test_loader = map_action_test_data_loader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

test_model(model, test_loader, device)


cuda


ValueError: Found input variables with inconsistent numbers of samples: [1, 11]

In [ ]:
# Print or use the metrics as needed
print(f'mAP: {mAP:.4f}')
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')